## Adult dataset preprocessing
This notebook is to preprocess the [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult)

Original processing code can be found [here](https://github.com/CDEIUK/bias-mitigation/blob/master/notebooks/finance/preprocessing.ipynb)

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "salary",
]

In [3]:
def clean_string(s):
    """
    Helper function that strips leading / trailing whitespace, lower
    cases, and replaces hyphens with underscores.
    """
    return s.strip().lower().replace("-", "_")


def parse_native_country(country):
    """
    Group countries other than United-States and Mexico into single
    "other" category"
    """
    country = clean_string(country)
    if country == "united_states" or country == "mexico":
        return country
    return "other"

In [24]:
train = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None,
        na_values=[" ?"],
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        salary=lambda df: (df.salary == " >50K").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [6]:
test = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        header=None,
        na_values=[" ?"],
        skiprows=1,
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        # note extra '.' in test set not present in train set
        salary=lambda df: (df.salary == " >50K.").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [7]:
test

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,25,private,11th,never_married,machine_op_inspct,own_child,black,1,0,0,40,united_states,0
1,38,private,hs_grad,married_civ_spouse,farming_fishing,husband,white,1,0,0,50,united_states,0
2,28,local_gov,assoc_acdm,married_civ_spouse,protective_serv,husband,white,1,0,0,40,united_states,1
3,44,private,some_college,married_civ_spouse,machine_op_inspct,husband,black,1,7688,0,40,united_states,1
4,34,private,10th,never_married,other_service,not_in_family,white,1,0,0,30,united_states,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,private,bachelors,never_married,prof_specialty,own_child,white,1,0,0,40,united_states,0
15056,39,private,bachelors,divorced,prof_specialty,not_in_family,white,0,0,0,36,united_states,0
15057,38,private,bachelors,married_civ_spouse,prof_specialty,husband,white,1,0,0,50,united_states,0
15058,44,private,bachelors,divorced,adm_clerical,own_child,asian_pac_islander,1,5455,0,40,united_states,0


In [8]:
assert set(train.education) == set(test.education)
assert set(train.race) == set(test.race)
assert set(train.relationship) == set(test.relationship)
assert set(train.marital_status) == set(test.marital_status)

In [14]:
one_hot_features = [
    "workclass",
    "education",
    "occupation",
    "race",
    "relationship",
    "marital_status",
    "native_country",
]

cts_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]

binary_features = ["sex", "salary"]

In [15]:
train["race"].value_counts()

white                 25933
black                  2817
asian_pac_islander      895
amer_indian_eskimo      286
other                   231
Name: race, dtype: int64

In [26]:
train_df = pd.concat(
    [train, pd.get_dummies(train.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

test_df = pd.concat(
    [test, pd.get_dummies(test.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

In [18]:
assert train_df.columns.tolist() == test_df.columns.tolist()

In [19]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [20]:
adult_data_dir = Path("/project/differential-privacy/adult_analysis/data/")

In [21]:
original_features = cts_features + one_hot_features + binary_features

train_df[original_features].to_csv(
    adult_data_dir / "train.csv", index=False
)
val_df[original_features].to_csv(
    adult_data_dir / "val.csv", index=False
)
test_df[original_features].to_csv(
    adult_data_dir / "test.csv", index=False
)

FileNotFoundError: [Errno 2] No such file or directory: '/project/differential-privacy/adult_analysis/data/train.csv'

In [22]:
ss = StandardScaler()

train_df[cts_features] = ss.fit_transform(train_df[cts_features])
val_df[cts_features] = ss.transform(val_df[cts_features])
test_df[cts_features] = ss.transform(test_df[cts_features])

/var/folders/99/zw2f_1t90kb48x4s3ygytnjc0000gn/T/ipykernel_26626/1747532109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[cts_features] = ss.fit_transform(train_df[cts_features])
/Users/umit/opt/anaconda3/envs/MLBDenv/lib/python3.8/site-packages/pandas/core/indexing.py:1737: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/var/folders/99/zw2f_1t90kb48x4s3ygytnjc0000gn/T/ipykernel_26626/1747532109.py:4: SettingWithCopyWarning: 
A value i

In [18]:
train_df.drop(columns=one_hot_features).to_csv(
    adult_data_dir / "train-one-hot.csv", index=False
)
val_df.drop(columns=one_hot_features).to_csv(
    adult_data_dir / "val-one-hot.csv", index=False
)
test_df.drop(columns=one_hot_features).to_csv(
    adult_data_dir / "test-one-hot.csv", index=False
)